# Trabajando con el paquete de csv y pandas para extraer datos de archivos CSV

Vamos a cargar y convertir la data que proviene de un archivo CSV comprimido.

In [1]:
"""Load & convert data from CSV file using Python built-in csv module"""
import bz2
import csv
from collections import namedtuple
from datetime import datetime

Column = namedtuple('Column', 'src dest convert')


def parse_timestamp(text):
    return datetime.strptime(text, '%Y-%m-%d %H:%M:%S')

## Aquí vamos indicando el nombre de la columna y el tipo de dato
## Usamos la función 'parse_time' para darle formato e indicar fecha

columns = [
    Column('VendorID', 'vendor_id', int),
    Column('passenger_count', 'num_passengers', int),
    Column('tip_amount', 'tip', float),
    Column('total_amount', 'price', float),
    Column('tpep_dropoff_datetime', 'dropoff_time', parse_timestamp),
    Column('tpep_pickup_datetime', 'pickup_time', parse_timestamp),
    Column('trip_distance', 'distance', float),
]

## Creación de un diccionario vacío para cada fila y posteriormente
## llenamos el diccionario con la información contenida en el archivo CSV

def iter_records(file_name):
    with bz2.open(file_name, 'rt') as fp:
        reader = csv.DictReader(fp)
        for csv_record in reader:
            record = {}
            for col in columns:
                value = csv_record[col.src]
                record[col.dest] = col.convert(value)
            yield record

## Vamos a imprimir en pantalla, las primeras 10 observaciones
## Con pprint podemos darle cierta apariencia agradable al resultado que aparece en pantalla
def example():
    from pprint import pprint

    for i, record in enumerate(iter_records('taxi.csv.bz2')):
        if i >= 10:
            break
        pprint(record)


example()


{'distance': 2.57,
 'dropoff_time': datetime.datetime(2018, 11, 1, 6, 43, 24),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 7, 10, 55),
 'price': 20.54,
 'tip': 4.74,
 'vendor_id': 2}
{'distance': 3.58,
 'dropoff_time': datetime.datetime(2018, 10, 31, 16, 50, 10),
 'num_passengers': 5,
 'pickup_time': datetime.datetime(2018, 10, 31, 16, 38, 25),
 'price': 13.8,
 'tip': 0.0,
 'vendor_id': 2}
{'distance': 2.39,
 'dropoff_time': datetime.datetime(2018, 10, 31, 20, 31, 47),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 20, 23, 41),
 'price': 11.3,
 'tip': 1.0,
 'vendor_id': 2}
{'distance': 0.5,
 'dropoff_time': datetime.datetime(2018, 10, 31, 22, 48, 28),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 22, 44, 24),
 'price': 5.8,
 'tip': 0.0,
 'vendor_id': 2}
{'distance': 1.81,
 'dropoff_time': datetime.datetime(2018, 10, 31, 23, 35, 30),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 23, 22, 18),

In [2]:
"""Load & converting data from CSV using Pandas"""
import pandas as pd

time_cols = ['tpep_dropoff_datetime', 'tpep_pickup_datetime']


def load_df(file_name):
    return pd.read_csv('taxi.csv.bz2', parse_dates=time_cols)


print(load_df('taxi.csv.bz2').head())

'''
def iter_df(file_name):
    yield from pd.read_csv(
        'taxi.csv.bz2', parse_dates=time_cols, chunksize=100)


for i, df in enumerate(iter_df('taxi.csv.bz2')):
    if i > 10:
        break
    print(len(df))
'''


   VendorID tpep_pickup_datetime  ... improvement_surcharge  total_amount
0         2  2018-10-31 07:10:55  ...                   0.3         20.54
1         2  2018-10-31 16:38:25  ...                   0.3         13.80
2         2  2018-10-31 20:23:41  ...                   0.3         11.30
3         2  2018-10-31 22:44:24  ...                   0.3          5.80
4         2  2018-10-31 23:22:18  ...                   0.3         13.56

[5 rows x 17 columns]


"\ndef iter_df(file_name):\n    yield from pd.read_csv(\n        'taxi.csv.bz2', parse_dates=time_cols, chunksize=100)\n\n\nfor i, df in enumerate(iter_df('taxi.csv.bz2')):\n    if i > 10:\n        break\n    print(len(df))\n"

In [3]:
# Si el tamaño del archivo es muy grande, podemos realizarle particiones a la lectura
# de tales archivos

def iter_df(file_name):
    yield from pd.read_csv(
        'taxi.csv.bz2', parse_dates=time_cols, chunksize=100)


for i, df in enumerate(iter_df('taxi.csv.bz2')):
    if i > 10:
        break
    print(len(df))

100
100
100
100
100
100
100
100
100
100
100
